# Joke Bot

This notebook demonstrates how to create a simple "joke bot" using the OpenAI Assistants API.

The notebook does the following:
1. Creates an assistant with instructions to tell jokes.
2. Creates a thread.
3. Asks the assistant to tell a joke.
4. Displays the assistant's response.

In [ ]:
import os
import json
from openai import OpenAI

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as env var>"))

assistant = client.beta.assistants.create(
    name="Joke Bot",
    instructions="You are a bot that tells funny jokes.",
    model="gpt-4o",
)
show_json(assistant)

In [ ]:
thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Tell me a joke.",
)

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)
show_json(run)

In [ ]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

run = wait_on_run(run, thread)

messages = client.beta.threads.messages.list(thread_id=thread.id)
show_json(messages)